In [1]:
# This is about getting the data[HTML page] of websites
import requests
topic_url='https://github.com/topics'
response=requests.get(topic_url)
print(response.status_code)

200


In [2]:
# Here we are getting all data into webpage.html page 
page_content=response.text
# print(page_content)
page_content[:1000]

with open("webpage.html","w+",encoding="utf-8") as f:
    f.write(page_content)
#print(page_content)

In [3]:
#After getting the data we have to parse[Structured way] using beautiful soup

from bs4 import BeautifulSoup
doc=BeautifulSoup(page_content,"html.parser")
#print(doc)


In [5]:
#To find how many topics present in the page[p tags]
# p_tags=doc.find_all('p')
# len(p_tags)


In [4]:
# Here we are taking all p tags and from p_tags we are getting all titles
selection_class="f3 lh-condensed mb-0 mt-1 Link--primary"
p_tags=doc.find_all("p",{"class":selection_class})
len(p_tags)

30

In [5]:
#T Getting all topic discrpitions

topic_discrption="f5 color-fg-muted mb-0 mt-1"
topic_discrption_tags=doc.find_all("p",{"class":topic_discrption})
len(topic_discrption_tags)
# topic_discrption_tags[:5]

30

In [6]:
# Getting all topic links 
topic_link_tags=doc.find_all('a',{"class":'no-underline flex-grow-0'})
len(topic_link_tags)
#print(topic_link_tags)
topic0_url="https://github.com"+topic_link_tags[0]['href']
print(topic0_url)

https://github.com/topics/3d


In [7]:
#Getting whole topic and keeping into the list
topic_lst=[]
for tags in p_tags:
    topic_lst.append(tags.text)
print(topic_lst[:5])

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android']


In [8]:
#Getting all topic descriptions and keeping into list
topic_desc=[]
for tags in topic_discrption_tags:
    topic_desc.append(tags.text)
topic_desc[:3]

['\n          3D modeling is the process of virtually developing the surface and structure of a 3D object.\n        ',
 '\n          Ajax is a technique for creating interactive web applications.\n        ',
 '\n          Algorithms are self-contained sequences that carry out a variety of tasks.\n        ']

In [9]:
# Getting all topic URL and keeping into list
topic_url=[]
base_url='https://github.com'
for tags in topic_link_tags:
    topic_url.append(base_url+tags['href'])
topic_url[5]

'https://github.com/topics/angular'

In [12]:
! pip install pandas


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# To create CSV files pandas has to download
import pandas as pd

In [11]:
# we are arranging the topictitles,topic descriptions and links in one row
# dictonaries is the best way to do 
topics_dict={
    'titles':p_tags,
     'descriptions':topic_discrption_tags,
     'urls':topic_url
}
# print(topics_dict)


In [12]:
#Here we are segretatting the data in the structural way using pandas
topics_df=pd.DataFrame(topics_dict)
# topics_desc

In [13]:
#creating CSV
topics_df.to_csv('topics.csv',index=None)
    

In [14]:
#Getting info about each and every topic
#Topic page url
topic_page_url=topic_url[1]
print(topic_page_url)
response=requests.get(topic_page_url)
response.status_code


https://github.com/topics/ajax


200

In [15]:
# souping the topic_docs
topic_doc=BeautifulSoup(response.text,'html.parser')
# topic_doc

In [16]:
# To get the No.of user and repo tags
h3_selection_class="f3 color-fg-muted text-normal lh-condensed"
repo_tags=topic_doc.find_all("h3",{"class":h3_selection_class})
len(repo_tags)
#repo_tags[0]

20

In [17]:
#user name
a_tags=repo_tags[0].find_all('a')
a_tags[0].text.strip()

'ljianshu'

In [18]:
#Repo name
a_tags[1].text.strip()


'Blog'

In [19]:
# To get repo url for the topic[3D]
base_url="htts://github.com"
repo_url=base_url+ a_tags[1]['href']
print(repo_url)

htts://github.com/ljianshu/Blog


In [20]:
#Stars tags
# stars_calss="Counter js-social-count"
span_tag="Counter js-social-count"
stars_tag=topic_doc.find_all('span',{"class":span_tag})
len(stars_tag)
# print(stars_tag)

20

In [21]:
stars_tag[0].text.strip()

'7.3k'

In [22]:
#Making a function which gives all stars counts
def parse_star_count (stars_tag):
    stars_tag=stars_tag.strip()
    if stars_tag[-1]=='k':
        return int(float(stars_tag[:-1])*1000)
    return int(stars_tag)
    

In [23]:
parse_star_count(stars_tag[0].text.strip())

7300

In [24]:
#Making a function which give username,reponame,stars count and url
def get_repo_info(h3_selection_class,stars_tag):
    #returns all info about repository
    a_tags=h3_selection_class.find_all('a')
    username=a_tags[0].text.strip()
    repo_name=a_tags[1].text.strip()
    repo_url=base_url+a_tags[1]['href']
    stars=parse_star_count(stars_tag.text.strip())
    return username,repo_name,stars,repo_url
    

In [28]:
get_repo_info(repo_tags[1],stars_tag[1])


('metafizzy',
 'infinite-scroll',
 7100,
 'htts://github.com/metafizzy/infinite-scroll')

In [25]:
#Getting username,reponame,stars and links in structrued way for 3D
topic_repo_dict={
    "username":[],
    "repo_name":[],
    "stars":[],
    "repo_url":[]
    }
for i in range(len(repo_tags)):
    repo_info=get_repo_info(repo_tags[i],stars_tag[i])
    topic_repo_dict["username"].append(repo_info[0])
    topic_repo_dict["repo_name"].append(repo_info[1])
    topic_repo_dict["stars"].append(repo_info[2])
    topic_repo_dict["repo_url"].append(repo_info[3])
    
    

In [30]:
# topic_repo_dict

In [26]:
#By using pandas library we will give the data in structured way
topic_repo_df=pd.DataFrame(topic_repo_dict)
# topic_repo_df

In [27]:
#Defining functions:

def get_topic_page(topics_url):
    #download the page
    topic_url='https://github.com/topics'
    response=requests.get(topics_url)    
    #check sucessfull response
    if response.status_code !=200:
        raise Exception("failed to raise load page {}".format(topic_url))
    #parse beautiful soup
    doc=BeautifulSoup(page_content,"html.parser")
    return topic_doc

#making a function which give username,reponame,stars count and url
def get_repo_info(h3_selection_class,stars_tag):
    #returns all info about repository
    a_tags=h3_selection_class.find_all('a')
    username=a_tags[0].text.strip()
    repo_name=a_tags[1].text.strip()
    repo_url=base_url+a_tags[1]['href']
    stars=parse_star_count(stars_tag.text.strip())
    return username,repo_name,stars,repo_url
    

def get_topic_repos(topic_doc):
    #get h3 tags
    h3_selection_class="f3 color-fg-muted text-normal lh-condensed"
    repo_tags=topic_doc.find_all("h3",{"class":h3_selection_class})
    #get stars tags
    span_tag="Counter js-social-count"
    stars_tag=topic_doc.find_all('span',{"class":span_tag})
    
    topic_repo_dict={
    "username":[],
    "repo_name":[],
    "stars":[],
    "repo_url":[]
    }
    
    #get repo_info
    for i in range(len(repo_tags)):
        repo_info=get_repo_info(repo_tags[i],stars_tag[i])
        topic_repo_dict["username"].append(repo_info[0])
        topic_repo_dict["repo_name"].append(repo_info[1])
        topic_repo_dict["stars"].append(repo_info[2])
        topic_repo_dict["repo_url"].append(repo_info[3])
    return pd.DataFrame(topic_repo_dict)
    


    
    
    

In [28]:
# get_topic_repos(get_topic_page(topic_url[0]))

In [41]:
# Making function to get url and name in csv files
def scrape_topic(topic_url,topic_name):
    topic_df=get_topic_repos(get_topic_page(topic_url))
    topic_df.to_csv(topic_name + '.csv',index=None)

In [1]:
# Defining all functons
# About topic titles
def get_topic_titles(doc):
    selection_class="f3 lh-condensed mb-0 mt-1 Link--primary"
    p_tags=doc.find_all("p",{"class":selection_class})
    topic_lst=[]
    for tags in p_tags:
        topic_lst.append(tags.text)
    return topic_lst
# About topic descriptions
def get_topic_descs(doc):
    topic_discrption_tags=doc.find_all("p",{"class":topic_discrption})
    topic_desc=[]
    for tags in topic_discrption_tags:
        topic_desc.append(tags.text.strip())
    return topic_desc
# About topic URL
def get_topic_urls(doc):
    topic_link_tags=doc.find_all('a',{"class":'no-underline flex-grow-0'})
    topic_url=[]
    base_url='https://github.com'
    for tags in topic_link_tags:
        topic_url.append(base_url+tags['href']) 
    return topic_url
# About topics
def scrape_topics():
    topic_url='https://github.com/topics'
    response=requests.get(topic_url)
    if response.status_code !=200:
        raise Exception("failed to raise load page {}".format(topic_url))
    topic_dict={
        'title':get_topic_titles(doc),
        'description':get_topic_descs(doc),
        'url':get_topic_urls(doc)
        }
    return pd.DataFrame(topic_dict)
    

In [45]:
# About getting all topic data in csv files
def scrape_topics_repo():
    topics_df=scrape_topics()
    for index,row in topics_df.iterrows():
        print('scrapping top repositories for "{}"'.format(row['title']))
        scrape_topic(row['url'],row['title'])    

In [47]:
# scrape_topics_repo()